# Cleaning raw images

The first thing to do with any CCD image is to clean the data from instrumental defects. For that, bias, dark and flat images should be used to clean the raw science images of the globular cluster M67. However, dark current of the selected images is negligible and, therefore, will not be used in the current exercise. Further information on most of the options and steps used in the current exercise can be found in the [Ccdproc reduction toolbox](https://ccdproc.readthedocs.io/en/latest/reduction_toolbox.html). 

As usual, required modules have to be imported first.

In [ ]:
import os.path
import warnings
from glob import glob

from astropy.wcs import FITSFixedWarning
import ccdproc

# FITS files are rather old and do not conform to FITS standard
warnings.filterwarnings("ignore", category=FITSFixedWarning)

%run plotfits.ipynb # plot() function is defined here

A notebook called [`plotfits.ipynb`](plotfits.ipynb) is imported here to plot the images. 

## Creating master bias

WFC images have an overscan region. For timing reasons, overscan will be 
assumed to be negligible and, therefore, only bias subtraction will be done. 

In [ ]:
raw_data_dir = os.path.join('..','resources', 'raw_data')
red_data_dir = os.path.join('..','resources', 'red_data')

bias_list = glob(os.path.join(raw_data_dir, 'bias*'))
master_bias_file = os.path.join(red_data_dir,'zero.fit')

if os.path.isfile(master_bias_file):
    os.remove(master_bias_file)

if not os.path.isdir(red_data_dir):
    os.mkdir(red_data_dir)

full_bias = ccdproc.combine(bias_list, output_file=master_bias_file, 
                            mem_limit=1e8, method='median', 
                            unit='adu', dtype='float32')

valid_area = slice(0, 4096), slice(53, 2101)
master_bias = ccdproc.trim_image(full_bias[valid_area])

plot(master_bias_file)

## Create master flat

All flats should be bias subtracted first. 

In [ ]:
flat_list = glob(os.path.join(raw_data_dir,'flat*'))

for flat in flat_list:
    raw_data = ccdproc.CCDData.read(flat, unit='adu')[valid_area]
    sub_data = ccdproc.subtract_bias(raw_data, master_bias)
    sub_data.write(os.path.join(red_data_dir, 'b_' + os.path.basename(flat)),
                   overwrite=True)

Now, all flats for Strömgren b filter can be combined **with the proper scaling**. 

In [ ]:
clean_flats = []
mean_values = []
master_flat_b_file = os.path.join(red_data_dir, 'master_flat_b.fit')

if os.path.isfile(master_flat_b_file):
    os.remove(master_flat_b_file)

for flat in glob(os.path.join(red_data_dir, 'b_flat_b*')):
    raw_data = ccdproc.CCDData.read(flat, unit='adu')
    mean = raw_data.data.mean()
    clean_flats += [raw_data]
    mean_values += [1/mean]
    print('Flat', flat, 'has mean value of', mean)

master_flat_b = ccdproc.combine(clean_flats, mem_limit=1e8, dtype='float32',
                                output_file=master_flat_b_file,
                                scale=mean_values, method='median')

plot(master_flat_b_file)

And the same for Strömgren y flats.

In [ ]:
clean_flats = []
mean_values = []
master_flat_y_file = os.path.join(red_data_dir, 'master_flat_y.fit')

if os.path.isfile(master_flat_y_file):
    os.remove(master_flat_y_file)

for flat in glob(os.path.join(red_data_dir, 'b_flat_y*')):
    raw_data = ccdproc.CCDData.read(flat, unit='adu')
    mean = raw_data.data.mean()
    clean_flats += [raw_data]
    mean_values += [1/mean]
    print('Flat', flat, 'has mean value of', mean)


master_flat_y = ccdproc.combine(clean_flats, mem_limit=1e8, dtype='float32',
                                output_file=master_flat_y_file,
                                scale=mean_values, method='median')

plot(master_flat_y_file)

## Processing science images

Once master bias and master flats have been obtained, science images can be cleaned from instrumental defects.

In [ ]:
for image in glob(os.path.join(raw_data_dir,'m67_b*')):
    raw_data = ccdproc.CCDData.read(image, unit='adu')[valid_area]
    clean_data = ccdproc.ccd_process(raw_data, master_bias=master_bias,
                                     master_flat=master_flat_b)
    output_file = os.path.join(red_data_dir, 'bf_' + os.path.basename(image))
    clean_data.write(output_file, overwrite=True)

for image in glob(os.path.join(raw_data_dir,'m67_y*')):
    raw_data = ccdproc.CCDData.read(image, unit='adu')[valid_area]
    clean_data = ccdproc.ccd_process(raw_data, master_bias=master_bias,
                                     master_flat=master_flat_y)
    output_file = os.path.join(red_data_dir, 'bf_' + os.path.basename(image))
    clean_data.write(output_file, overwrite=True)

plot(image)
plot(output_file)